#CHAPTER 6 : Data Loading, Storage, and File Formats

##Reading and Writing Data in Text Format

In [5]:
import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10, 6))
pd.options.display.max_colwidth = 75
pd.options.display.max_columns = 20
np.set_printoptions(precision=4, suppress=True)

In [ ]:
!cat examples/ex1.csv

In [ ]:
df = pd.read_csv("examples/ex1.csv")
df

In [ ]:
!cat examples/ex2.csv

In [ ]:
pd.read_csv("examples/ex2.csv", header=None)
pd.read_csv("examples/ex2.csv", names=["a", "b", "c", "d", "message"])

In [ ]:
names = ["a", "b", "c", "d", "message"]
pd.read_csv("examples/ex2.csv", names=names, index_col="message")

In [ ]:
!cat examples/csv_mindex.csv
parsed = pd.read_csv("examples/csv_mindex.csv",
                     index_col=["key1", "key2"])
parsed

In [7]:
!cat examples/ex3.txt

cat: examples/ex3.txt: No such file or directory


In [ ]:
result = pd.read_csv("examples/ex3.txt", sep="\s+")
result

In [ ]:
!cat examples/ex4.csv
pd.read_csv("examples/ex4.csv", skiprows=[0, 2, 3])

In [ ]:
!cat examples/ex5.csv
result = pd.read_csv("examples/ex5.csv")
result

In [ ]:
pd.isna(result)

In [ ]:
result = pd.read_csv("examples/ex5.csv", na_values=["NULL"])
result

In [ ]:
result2 = pd.read_csv("examples/ex5.csv", keep_default_na=False)
result2
result2.isna()
result3 = pd.read_csv("examples/ex5.csv", keep_default_na=False,
                      na_values=["NA"])
result3
result3.isna()

In [ ]:
sentinels = {"message": ["foo", "NA"], "something": ["two"]}
pd.read_csv("examples/ex5.csv", na_values=sentinels,
            keep_default_na=False)

###Reading Text Files in Pieces

In [ ]:
pd.options.display.max_rows = 10

In [ ]:
result = pd.read_csv("examples/ex6.csv")
result

In [ ]:
pd.read_csv("examples/ex6.csv", nrows=5)

In [ ]:
chunker = pd.read_csv("examples/ex6.csv", chunksize=1000)
type(chunker)

In [ ]:
chunker = pd.read_csv("examples/ex6.csv", chunksize=1000)

tot = pd.Series([], dtype='int64')
for piece in chunker:
    tot = tot.add(piece["key"].value_counts(), fill_value=0)

tot = tot.sort_values(ascending=False)

In [ ]:
tot[:10]

###Writing Data to Text Format

In [ ]:
data = pd.read_csv("examples/ex5.csv")
data

In [ ]:
data.to_csv("examples/out.csv")
!cat examples/out.csv

In [ ]:
import sys
data.to_csv(sys.stdout, sep="|")

In [ ]:
data.to_csv(sys.stdout, na_rep="NULL")

In [ ]:
data.to_csv(sys.stdout, index=False, header=False)

In [ ]:
data.to_csv(sys.stdout, index=False, columns=["a", "b", "c"])

###Working with Other Delimited Formats

In [9]:
!cat examples/ex7.csv

cat: examples/ex7.csv: No such file or directory


In [ ]:
import csv
f = open("examples/ex7.csv")
reader = csv.reader(f)

In [ ]:
for line in reader:
    print(line)
f.close()

In [ ]:
with open("examples/ex7.csv") as f:
    lines = list(csv.reader(f))

In [ ]:
header, values = lines[0], lines[1:]

In [ ]:
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

###JSON Data

In [ ]:
obj = """
{"name": "Wes",
 "cities_lived": ["Akron", "Nashville", "New York", "San Francisco"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 34, "hobbies": ["guitars", "soccer"]},
              {"name": "Katie", "age": 42, "hobbies": ["diving", "art"]}]
}
"""

In [ ]:
import json
result = json.loads(obj)
result

In [ ]:
asjson = json.dumps(result)
asjson

In [ ]:
siblings = pd.DataFrame(result["siblings"], columns=["name", "age"])
siblings

In [ ]:
!cat examples/example.json

In [ ]:
data = pd.read_json("examples/example.json")
data

In [ ]:
data.to_json(sys.stdout)
data.to_json(sys.stdout, orient="records")

###XML and HTML: Web Scraping

In [ ]:
tables = pd.read_html("examples/fdic_failed_bank_list.html")
len(tables)
failures = tables[0]
failures.head()

In [ ]:
close_timestamps = pd.to_datetime(failures["Closing Date"])
close_timestamps.dt.year.value_counts()

###Parsing XML with lxml.objectify

In [ ]:
from lxml import objectify

path = "datasets/mta_perf/Performance_MNR.xml"
with open(path) as f:
    parsed = objectify.parse(f)
root = parsed.getroot()

In [ ]:
data = []

skip_fields = ["PARENT_SEQ", "INDICATOR_SEQ",
               "DESIRED_CHANGE", "DECIMAL_PLACES"]

for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

In [ ]:
perf = pd.DataFrame(data)
perf.head()

In [ ]:
perf2 = pd.read_xml(path)
perf2.head()

##Binary Data Formats

In [ ]:
frame = pd.read_csv("examples/ex1.csv")
frame
frame.to_pickle("examples/frame_pickle")

In [ ]:
pd.read_pickle("examples/frame_pickle")

In [ ]:
!rm examples/frame_pickle

In [ ]:
fec = pd.read_parquet('datasets/fec/fec.parquet')

###Reading Microsoft Excel Files

In [ ]:
xlsx = pd.ExcelFile("examples/ex1.xlsx")

In [ ]:
xlsx.sheet_names

In [ ]:
xlsx.parse(sheet_name="Sheet1")

In [ ]:
xlsx.parse(sheet_name="Sheet1", index_col=0)

In [ ]:
frame = pd.read_excel("examples/ex1.xlsx", sheet_name="Sheet1")
frame

In [ ]:
writer = pd.ExcelWriter("examples/ex2.xlsx")
frame.to_excel(writer, "Sheet1")
writer.close()

In [ ]:
frame.to_excel("examples/ex2.xlsx")

###Using HDF5 Format

In [ ]:
!rm examples/ex2.xlsx

In [ ]:
!rm -f examples/mydata.h5

In [ ]:
frame = pd.DataFrame({"a": np.random.standard_normal(100)})
store = pd.HDFStore("examples/mydata.h5")
store["obj1"] = frame
store["obj1_col"] = frame["a"]
store

In [ ]:
store["obj1"]

In [ ]:
store.put("obj2", frame, format="table")
store.select("obj2", where=["index >= 10 and index <= 15"])
store.close()

In [ ]:
frame.to_hdf("examples/mydata.h5", "obj3", format="table")
pd.read_hdf("examples/mydata.h5", "obj3", where=["index < 5"])

In [ ]:
import os
os.remove("examples/mydata.h5")

##Interacting with Web APIs

In [10]:
import requests
url = "https://api.github.com/repos/pandas-dev/pandas/issues"
resp = requests.get(url)
resp.raise_for_status()
resp

<Response [200]>

In [11]:
data = resp.json()
data[0]["title"]

'Add failing test for pyarrow datetime merge issue'

In [12]:
issues = pd.DataFrame(data, columns=["number", "title",
                                     "labels", "state"])
issues

,number,title,labels,state
0,62381,Add failing test for pyarrow datetime merge issue,[],open
1,62380,ENH: set __module__ on IndexSlice,[],open
2,62379,DOC: Link `pandas.merge` API page to User Guide (:ref:`merging`),[],open
3,62376,BUG: Raise TypeError for mismatched signed/unsigned dtypes in IntervalI...,[],open
4,62375,[backport 2.3.x] CoW: disable chained assignment detection for Python 3...,[],open
5,62374,TST: test that re-using a list of groupers will not throw ValueError,[],open
6,62372,TST: groupby.sum with large integers,[],open
7,62371,BUG: handle None in dtype=bool Series with Kleene 3-valued logic (#62370),[],open
8,62370,BUG: None in dtype=bool Series does not behave like 3-valued logic,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ==', 'url': 'https://api.g...",open
9,62369,API: setitem-with-expansion casting,"[{'id': 1741841389, 'node_id': 'MDU6TGFiZWwxNzQxODQxMzg5', 'url': 'http...",open


##Interacting with Databases

In [13]:
import sqlite3

query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
 c REAL,        d INTEGER
);"""

con = sqlite3.connect("mydata.sqlite")
con.execute(query)
con.commit()

In [14]:
data = [("Atlanta", "Georgia", 1.25, 6),
        ("Tallahassee", "Florida", 2.6, 3),
        ("Sacramento", "California", 1.7, 5)]
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"

con.executemany(stmt, data)
con.commit()

In [15]:
cursor = con.execute("SELECT * FROM test")
rows = cursor.fetchall()
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

In [16]:
cursor.description
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


In [17]:
import sqlalchemy as sqla
db = sqla.create_engine("sqlite:///mydata.sqlite")
pd.read_sql("SELECT * FROM test", db)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


In [18]:
!rm mydata.sqlite

#Kết Thúc